In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
import cv2
import numpy as np
import random
import matplotlib.pyplot as plt
import tensorflow as tf
from keras import backend as K
from keras.models import Model, Sequential
from keras.layers import *
from keras.callbacks import ModelCheckpoint
from keras.applications import InceptionResNetV2

In [ ]:
dir = "/content/drive/My Drive/Attendence_system/trainset"
batchsize = 32
image_size = 96

In [ ]:
def data(dir):
  i = 0 
  for subdir in os.listdir(dir):
    subdir_path = dir + '/' + subdir
    for imagedir in os.listdir(subdir_path):
      print(imagedir)
      imagedir_path = subdir_path + '/' + imagedir
      for image in os.listdir(imagedir_path):
        face_extraction(imagedir_path+'/'+image,imagedir_path,image,imagedir)
  return 


In [ ]:
def face_extraction(original_image,imagedir_path,image_name,imagedir):
  image = cv2.imread(original_image)
  gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

  faceCascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
  faces = faceCascade.detectMultiScale(gray,scaleFactor=1.3,minNeighbors=6)
  if len(faces) != 1:
    return 0
  x,y,w,h = faces.squeeze()
  cropImage = image[y:y+h,x:x+w]

  if "script" in original_image:
    path = f"/content/drive/My Drive/Attendence_system/Face_extracted/{imagedir}_anchor.jpg"
  else:
    path = f"/content/drive/My Drive/Attendence_system/Face_extracted/{imagedir}_positive.jpg"
  cv2.imwrite(path,cropImage)
  
  return cropImage

In [ ]:
def image_preprocess(image_path):
 # print(image_path)
  image = cv2.imread(image_path)
  resizeImage =cv2.resize(image,(image_size,image_size))
  resizeImage=resizeImage.astype('float32')/255.0
  return resizeImage

In [ ]:
def get_data(batch_size):

  a =[]
  p = []
  n = []
  path = "/content/drive/My Drive/Attendence_system/Face_extracted"
  listofImage = list(os.listdir(path))
  listofImage.remove('.ipynb_checkpoints')
  i = 0
  while i < batch_size:

    randomIndex= int(random.random()*len(listofImage))
    #print(randomIndex)
    if randomIndex == (len(listofImage)):
      randomIndex = randomIndex - 1
    
    getImage = listofImage[randomIndex]
    anchorImage = getImage.split("_")[0]+'_'+getImage.split("_")[1]+"_anchor.jpg"
    positiveImage = getImage.split("_")[0]+'_'+getImage.split("_")[1]+"_positive.jpg"
    if anchorImage in listofImage and positiveImage in listofImage:
      a.append(path+'/'+anchorImage)
      p.append(path+'/'+positiveImage)
    else:
      continue
        
    nRandomIndex = int(random.random()*len(listofImage))
    while nRandomIndex == randomIndex:
      nRandomIndex = int(random.random()*len(listofImage))
    i = i + 1

    n.append(path+'/'+listofImage[nRandomIndex])

  return a,p,n

In [ ]:
def data_generation(batchsize):

  faceAnchor = np.zeros((batchsize,image_size,image_size,3))
  facepositive = np.zeros((batchsize,image_size,image_size,3))
  facenagative = np.zeros((batchsize,image_size,image_size,3))
  a,p,n  = get_data(batchsize)

  for i in range(len(a)):
    faceAnchor[i:] = image_preprocess(a[i])
    facepositive[i:] = image_preprocess(p[i])
    facenagative[i:] = image_preprocess(n[i])

  x = [faceAnchor , facepositive , facenagative]
  y = np.zeros((batchsize,1)).astype("float32")
  return (x,y)

In [ ]:
(x,y) = data_generation(3200)

In [ ]:
def triplet_loss(y_true, y_pred, alpha = 0.2):    
    anchor,position,nagative=y_pred[:,0:128],y_pred[:,128:256],y_pred[:,256:384]
    
    pos_dist =tf.reduce_sum(tf.square(tf.subtract(anchor,position)),axis=-1)
    neg_dist =  tf.reduce_sum(tf.square(tf.subtract(anchor,nagative)),axis = -1)
    basic_loss = tf.add(tf.subtract(pos_dist,neg_dist),alpha)
    loss = tf.reduce_sum(tf.maximum(basic_loss,0.0),axis = None)
    
    return loss

In [ ]:
def faceRecogizationModel(inputshape = (96,96,3)):
  model = Sequential()
  model.add(InceptionResNetV2(input_shape=(96,96,3), include_top = False, weights='imagenet'))
  #model.add(AveragePooling2D())
  model.layers[0].trainable = True
  model.add(Flatten())
  model.add(Dense(128))
  model.add(Lambda(lambda  x: K.l2_normalize(x,axis=1)))
  return model


In [ ]:
FRmodel = faceRecogizationModel(inputshape=(96,96,3))
#FRmodel.build(input_shape=(96,96,3))
#FRmodel.compile(optimizer = 'adam', loss = triplet_loss, metrics = ['accuracy'])

In [ ]:
FRmodel.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
inception_resnet_v2 (Functio (None, 1, 1, 1536)        54336736  
_________________________________________________________________
flatten_4 (Flatten)          (None, 1536)              0         
_________________________________________________________________
dense_4 (Dense)              (None, 128)               196736    
_________________________________________________________________
lambda_4 (Lambda)            (None, 128)               0         
Total params: 54,533,472
Trainable params: 54,472,928
Non-trainable params: 60,544
_________________________________________________________________


In [ ]:
anchorInput = Input((96,96,3))
positiveInput = Input((96,96,3))
nagativeInput = Input((96,96,3))
Output = concatenate([FRmodel(anchorInput), FRmodel(positiveInput), FRmodel(nagativeInput)])
triplet_model = Model(inputs = [anchorInput , positiveInput , nagativeInput] , outputs = Output)

In [ ]:
triplet_model.compile(optimizer='adam',loss=triplet_loss)

In [ ]:
triplet_model.summary()

Model: "functional_9"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_18 (InputLayer)           [(None, 96, 96, 3)]  0                                            
__________________________________________________________________________________________________
input_19 (InputLayer)           [(None, 96, 96, 3)]  0                                            
__________________________________________________________________________________________________
input_20 (InputLayer)           [(None, 96, 96, 3)]  0                                            
__________________________________________________________________________________________________
sequential_4 (Sequential)       (None, 128)          54533472    input_18[0][0]                   
                                                                 input_19[0][0]        

In [ ]:
triplet_model.fit(x,y,epochs = 10, batch_size= 32)

Epoch 1/10
100/100 [==============================] - 34s 338ms/step - loss: 3.0664
Epoch 2/10
100/100 [==============================] - 34s 342ms/step - loss: 1.1075
Epoch 3/10
100/100 [==============================] - 35s 347ms/step - loss: 0.9062
Epoch 4/10
100/100 [==============================] - 35s 350ms/step - loss: 0.7084
Epoch 5/10
100/100 [==============================] - 35s 352ms/step - loss: 0.5307
Epoch 6/10
100/100 [==============================] - 35s 354ms/step - loss: 0.5391
Epoch 7/10
100/100 [==============================] - 35s 355ms/step - loss: 0.4598
Epoch 8/10
100/100 [==============================] - 36s 356ms/step - loss: 0.3515
Epoch 9/10
100/100 [==============================] - 35s 355ms/step - loss: 0.2324
Epoch 10/10
100/100 [==============================] - 35s 354ms/step - loss: 0.2156


In [ ]:
triplet_model.save('/content/drive/My Drive/Attendence_system/AttendenceSystem.h5')